# Preparation for the moodle exercise in Spark

In this jupyter notebook we are going to make the preprocessing part of the dataset that is going to be used in the graded exercise of this week.
1. Change to exercise08 repository

2. Start docker <br>
docker-compose up -d

3. Getting the data:
Follow the procedure that is described below. The dataset can be found here: http://data.greatlanguagegame.com.s3.amazonaws.com/confusion-2014-03-02.tbz2. More specifically do the following:
- download the data      :<br> ```wget http://data.greatlanguagegame.com.s3.amazonaws.com/confusion-2014-03-02.tbz2```
- extract the data       :<br> ```tar -jxvf confusion-2014-03-02.tbz2```

4. copy the data to hdfs :<br> ```docker cp confusion-2014-03-02/confusion-2014-03-02.json jupyter:/home/jovyan/work``` <br>
(Copying the data to hdfs needs to be done only once and it might take 1-2 minutes.)

For more information about the dataset, you can refer to https://lars.yencken.org/datasets/great-language-game/

## Preprocessing commands
In your newly created notebook run these commands in order to have the dataset into an RDD:

In [1]:
import json
from pyspark.sql import SparkSession
from pyspark import SparkConf

spark = SparkSession.builder.master('local').getOrCreate()
sc = spark.sparkContext

path = "confusion-2014-03-02.json"
raw_data = sc.textFile(path)
dataset = raw_data.map(json.loads).cache()

After that you will be able to run the queries of the moodle question of this week. The RDD that you have to perform your queries on is the ```dataset``` one. For example, the following command returns one element of the dataset:

In [2]:
dataset.take(1)

[{'guess': 'Norwegian',
  'target': 'Norwegian',
  'country': 'AU',
  'choices': ['Maori', 'Mandarin', 'Norwegian', 'Tongan'],
  'sample': '48f9c924e0d98c959d8a6f1862b3ce9a',
  'date': '2013-08-19'}]

Find the number of games where the guessed language is correct (meaning equal to the target one) and that language is Russian.

In [4]:
dataset.filter(lambda x: x["guess"]==x["target"] and x["target"]=="Russian").count()

290818

Return the number of distinct "target" languages.

In [5]:
dataset.map(lambda x: x["target"]).distinct().count()

78

Return the sample IDs (i.e., the sample field) of the top three games where the guessed language is correct (equal to the target one) ordered by language (ascending), then by country (ascending), then by date (ascending)

In [11]:
d1 = dataset.filter(lambda x: x["guess"]==x["target"])
d2 = d1.sortBy(lambda x: (x["target"], x["country"], x["date"]))
d3 = d2.map(lambda x: x["sample"])
d3.take(3)

['00b85faa8b878a14f8781be334deb137',
 'efcd813daec1c836d9f030b30caa07ce',
 '13722ceed1eede7ba597ade9b4cb9807']

Aggregate all games by country and target language, counting the number of guesses for each group and return the frequencies of the three most frequent country/language combinations.

In [17]:
dataset.map(lambda x: ((x["country"], x["target"]), 1)).reduceByKey(lambda x,y: x+y).sortBy(lambda x: x[1], ascending=False).take(3)

[(('US', 'French'), 112934),
 (('US', 'German'), 112007),
 (('US', 'Spanish'), 110919)]

Find the percentage of games where (the answer was correct && the correct guess was the first choice amongst the array of possible answers)

In [15]:
correct_and_first = dataset.filter(lambda x: x["guess"]==x["target"] and x["guess"]==x["choices"][0]).count()
total = dataset.count()
correct_and_first/total

0.25603983084476356

Sort the languages by decreasing overall percentage of correct guesses and return the first three languages.

In [18]:
dataset.filter(lambda x: x["guess"]==x["target"]).map(lambda x: (x["target"], 1)).reduceByKey(lambda x,y: x+y).sortBy(lambda x: x[1], ascending=False).take(3)

[('French', 325430), ('German', 315271), ('Spanish', 304147)]

Return the number of games played on the latest day.

In [22]:
d1 = dataset.sortBy(lambda x: x["date"], ascending=False)
last_day = d1.take(1)
print(last_day)
dataset.filter(lambda x: x["date"] == last_day).count()

KeyboardInterrupt: 